# **Customer Retention & Churn Analysis (RavenStack SaaS Dataset)**

## **1. Business Understanding**

The objective of this project is to analyze SaaS customer and subscription data to answer critical retention and churn questions that business teams rely on to drive revenue growth and sustainable customer engagement. Using the RavenStack dataset this analysis evaluates historical customer activity, subscription records, feature usage, support interactions, and churn events to understand patterns that influence customer retention.

Rather than focusing only on descriptive statistics, this project is guided by four key business questions:

1. Why are customers leaving the platform?  
2. Which customer segments are most likely to churn?  
3. How long do customers typically stay active?  
4. What actions can improve customer retention?

The dataset includes information on customer accounts, subscription plans, churn events, feature engagement, and support tickets. Initial review indicates variation across plan tiers, industries, and customer behavior, suggesting opportunities to optimize customer engagement, support processes, and product feature adoption.

This analysis aims to transform raw customer and subscription data into actionable insights that can support product, growth, and retention decision-making.

### **Key Factors Influencing Customer Retention**

**Churn Reasons**  
Understanding why customers leave is critical for targeted retention strategies. Early exploration shows multiple churn drivers including pricing concerns, support dissatisfaction, and unmet product needs.

**Customer Segmentation**  
Retention differs across plan tiers, industries, and countries. Segment-level analysis allows the business to identify high-risk cohorts and tailor interventions.

**Tenure and Engagement**  
Customer lifetime varies widely depending on subscription plan, feature usage, and engagement patterns. Measuring tenure and usage metrics is essential to forecast churn risk.

**Growth Opportunity Areas**  
By combining churn reasons, segment analysis, tenure, and engagement, the business can pinpoint actions to improve retention and increase lifetime value.

### **Stakeholders**

- Product Managers responsible for feature development and engagement.  
- Growth & Retention Teams monitoring churn trends and retention campaigns.  
- Customer Support Teams improving resolution times and satisfaction.  
- Executive Leadership using insights to guide strategic decisions and resource allocation.

### **Problem Statement**

SaaS businesses often lack a clear understanding of why customers churn and which segments are at the highest risk. Without this insight, retention initiatives may be misdirected, reducing overall revenue potential.  

This project seeks to answer the key business questions around churn drivers, segment risk, customer lifetime, and engagement patterns using historical subscription and customer data to support informed decision-making.

### **Objectives**

1. Identify the primary reasons customers leave the platform.  
2. Analyze which customer segments are most likely to churn.  
3. Measure how long customers stay active and engaged.  
4. Provide actionable recommendations to improve customer retention.

### **Metrics of Success**

The project will be considered successful if:  

- Each key business question is clearly answered using data and visual analysis.  
- Insights are supported by accurate metrics such as churn rate, tenure, usage counts, and satisfaction scores.  
- Recommendations are actionable and aligned with the analysis findings.  
- Results are communicated clearly for both technical and non-technical stakeholders.

---

## **2. Data Understanding**

To analyze customer retention and churn, this project uses the **RavenStack SaaS Dataset** obtained from Kaggle [https://www.kaggle.com/datasets/rivalytics/saas-subscription-and-churn-analytics-dataset?utm_source=chatgpt.com]. The dataset is synthetic but reflects real SaaS behavior, including accounts, subscriptions, feature usage, support tickets, and churn events.

The dataset includes five CSV files:

- `accounts.csv` (500 records) – Customer account information including industry, plan, country, signup date, and churn flag.  
- `subscriptions.csv` (5,000 records) – Subscription details including start/end dates, plan tier, seats, MRR/ARR, and churn status.  
- `feature_usage.csv` (25,000 records) – Feature engagement data per subscription including usage counts, duration, and beta features.  
- `support_tickets.csv` (2,000 records) – Customer support interactions including resolution time, priority, and satisfaction scores.  
- `churn_events.csv` (600 records) – Churn instances including reasons, refunds, and feedback.

### **Dataset Overview**

| Attribute | Description |
|-----------|-------------|
| Number of records | Accounts: 500, Subscriptions: 5,000, Feature Usage: 25,000, Support Tickets: 2,000, Churn Events: 600 |
| Data types | Dates, categorical (industry, plan, reason), numeric (usage counts, tenure, MRR, ARR), boolean (churn, trial) |
| Missing values | Some feedback_text, satisfaction_score, end_date for active subscriptions |

### **Feature Description**

- `account_id`: Unique customer identifier  
- `account_name`: Fictional company name  
- `industry`: SaaS vertical (e.g., DevTools, EdTech)  
- `country`: Country code of the account  
- `signup_date`: Customer creation date  
- `plan_tier`: Basic, Pro, or Enterprise  
- `seats`: Licensed user count  
- `churn_flag`: Indicates if account churned  
- `subscription_id`: Unique subscription record  
- `start_date` / `end_date`: Subscription start and end dates  
- `mrr_amount` / `arr_amount`: Monthly and annual recurring revenue  
- `usage_count`, `usage_duration_secs`: Feature engagement metrics  
- `ticket_id`, `resolution_time_hours`, `satisfaction_score`: Support metrics  
- `churn_date`, `reason_code`, `feedback_text`: Churn event details  

### **Data Relevance**

This dataset is highly relevant for the churn analysis task because its features directly support the key business questions:

- **Why are customers leaving the platform?**; Analyzed using `churn_events.reason_code` and `feedback_text`.  
- **Which customer segments are most likely to churn?**; Evaluated using `accounts.plan_tier`, `industry`, `country`, and `seats`.  
- **How long do customers typically stay active?**; Measured using `subscriptions.start_date` and `subscriptions.end_date` to calculate tenure.  
- **What actions can improve retention?**; Explored through `feature_usage` engagement metrics and `support_tickets` satisfaction/resolution metrics.

Overall, the dataset provides **comprehensive, customer-level, and subscription-level information** needed to perform retention and churn analysis. It supports segment analysis, lifetime calculation, engagement assessment, and actionable insight generation, making it well-suited for SaaS customer retention analysis.

In [387]:
import pandas as pd 
import numpy as np
from datetime import datetime

In [388]:
# Load CSVs
accounts = pd.read_csv("ravenstack_accounts.csv")
churn_events = pd.read_csv("ravenstack_churn_events.csv")
feature_usage = pd.read_csv("ravenstack_feature_usage.csv")
subscriptions = pd.read_csv("ravenstack_subscriptions.csv")
support_tickets = pd.read_csv("ravenstack_support_tickets.csv")


# Quick inspection
print(accounts.head())
print(churn_events.head())
print(feature_usage.head())
print(subscriptions.head())
print(support_tickets.head())

  account_id account_name    industry country signup_date referral_source  \
0   A-2e4581    Company_0      EdTech      US  2024-10-16         partner   
1   A-43a9e3    Company_1     FinTech      IN  2023-08-17           other   
2   A-0a282f    Company_2    DevTools      US  2024-08-27         organic   
3   A-1f0ac7    Company_3  HealthTech      UK  2023-08-27           other   
4   A-ce550d    Company_4  HealthTech      US  2024-10-27           event   

    plan_tier  seats  is_trial  churn_flag  
0       Basic      9     False       False  
1       Basic     18     False        True  
2       Basic      1     False       False  
3       Basic     24      True       False  
4  Enterprise     35     False        True  
  churn_event_id account_id  churn_date reason_code  refund_amount_usd  \
0       C-816288   A-c37cab  2024-10-27     pricing               4.03   
1       C-5a81e7   A-37f969  2024-06-25     support              96.45   
2       C-a174be   A-b07346  2024-11-12      

In [389]:
print(accounts.info())
print(churn_events.info())
print(feature_usage.info())
print(subscriptions.info())
print(support_tickets.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account_id       500 non-null    object
 1   account_name     500 non-null    object
 2   industry         500 non-null    object
 3   country          500 non-null    object
 4   signup_date      500 non-null    object
 5   referral_source  500 non-null    object
 6   plan_tier        500 non-null    object
 7   seats            500 non-null    int64 
 8   is_trial         500 non-null    bool  
 9   churn_flag       500 non-null    bool  
dtypes: bool(2), int64(1), object(7)
memory usage: 32.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   churn_event_id            600 non-null    object 
 1   account_id      

In [390]:
print(accounts.isna().sum())
print(churn_events.isna().sum())
print(feature_usage.isna().sum())
print(subscriptions.isna().sum())
print(support_tickets.isna().sum())

account_id         0
account_name       0
industry           0
country            0
signup_date        0
referral_source    0
plan_tier          0
seats              0
is_trial           0
churn_flag         0
dtype: int64
churn_event_id                0
account_id                    0
churn_date                    0
reason_code                   0
refund_amount_usd             0
preceding_upgrade_flag        0
preceding_downgrade_flag      0
is_reactivation               0
feedback_text               148
dtype: int64
usage_id               0
subscription_id        0
usage_date             0
feature_name           0
usage_count            0
usage_duration_secs    0
error_count            0
is_beta_feature        0
dtype: int64
subscription_id         0
account_id              0
start_date              0
end_date             4514
plan_tier               0
seats                   0
mrr_amount              0
arr_amount              0
is_trial                0
upgrade_flag            0
dow

Observation:
#### Accounts Table
- No missing values detected.
- Account-level data is clean and complete.
- Plan tier, industry, country, and churn_flag are fully populated.
- This table is reliable for segmentation analysis.
#### Churn vents Table
- `feedback_text` has 148 missing values
- All other columns are complete
- Not all churned customers provided written feedback. This is realistic in SaaS many users cancel without detailed explanation.
- Since reason_code has no missing values, we can still analyze churn drivers reliably.
#### Feature Usage Table
- No missing values detected.
- Usage metrics are complete.
- Engagement analysis (usage_count, duration, errors, beta features) is reliable.
- This dataset will strongly support engagement vs churn analysis.
#### Subscription table
- `end_date` has 4514 missing values
- Out of 5000 subscriptions: Only 486 have end_date. 4514 are still active.
- This is expected behavior, not a data problem.

Missing end_date means:
 - Subscription is still active
 - Customer has not churned
#### Support Tickets Table
- `satisfaction_score` has 825 missing
- Out of 2000 tickets: 1175 have satisfaction ratings 825 do not
- Not all customers leave feedback after support interaction.This is very realistic.


In [391]:
print(accounts.columns)
print(churn_events.columns)
print(feature_usage.columns)
print(subscriptions.columns)
print(support_tickets.columns)

Index(['account_id', 'account_name', 'industry', 'country', 'signup_date',
       'referral_source', 'plan_tier', 'seats', 'is_trial', 'churn_flag'],
      dtype='object')
Index(['churn_event_id', 'account_id', 'churn_date', 'reason_code',
       'refund_amount_usd', 'preceding_upgrade_flag',
       'preceding_downgrade_flag', 'is_reactivation', 'feedback_text'],
      dtype='object')
Index(['usage_id', 'subscription_id', 'usage_date', 'feature_name',
       'usage_count', 'usage_duration_secs', 'error_count', 'is_beta_feature'],
      dtype='object')
Index(['subscription_id', 'account_id', 'start_date', 'end_date', 'plan_tier',
       'seats', 'mrr_amount', 'arr_amount', 'is_trial', 'upgrade_flag',
       'downgrade_flag', 'churn_flag', 'billing_frequency', 'auto_renew_flag'],
      dtype='object')
Index(['ticket_id', 'account_id', 'submitted_at', 'closed_at',
       'resolution_time_hours', 'priority', 'first_response_time_minutes',
       'satisfaction_score', 'escalation_flag'],
  

### Data Preparation
**Data Cleaning**

In [392]:
# Convert date columns
# Subscriptions
subscriptions['start_date'] = pd.to_datetime(subscriptions['start_date'])
subscriptions['end_date'] = pd.to_datetime(subscriptions['end_date'])

# Accounts
accounts['signup_date'] = pd.to_datetime(accounts['signup_date'])

# Churn events
churn_events['churn_date'] = pd.to_datetime(churn_events['churn_date'])

# Feature usage
feature_usage['usage_date'] = pd.to_datetime(feature_usage['usage_date'])

# Support tickets
support_tickets['submitted_at'] = pd.to_datetime(support_tickets['submitted_at'])
support_tickets['closed_at'] = pd.to_datetime(support_tickets['closed_at'])

In [393]:
# Aggregate feature usage per subbscription
usage_summary = feature_usage.groupby('subscription_id').agg({
    'usage_count':'sum',
    'usage_duration_secs':'sum',
    'error_count':'sum',
    'is_beta_feature':'sum'  
}).reset_index()

In [394]:
# Aggregate support tickets per account
ticket_summary = support_tickets.groupby('account_id').agg({
    'ticket_id':'count',               
    'resolution_time_hours':'mean',    
    'satisfaction_score':'mean'}).reset_index().rename(columns={'ticket_id':'total_tickets'})

In [395]:
# Aggregate churn events per account
churn_summary = churn_events.groupby('account_id').agg({
    'reason_code':'first',      # first churn reason
    'feedback_text':'first'     # first feedback
}).reset_index()


In [396]:
# Merge subscriptions with accounts
master_df = subscriptions.merge(
    accounts[['account_id','industry','country','plan_tier','seats','signup_date','referral_source','is_trial']],
    on='account_id',
    how='left'
)

In [397]:
# Aggregate feature usage
master_df = master_df.merge(usage_summary, on='subscription_id', how='left')

# Fill missing usage with 0 for subscriptions with no usage
master_df[['usage_count','usage_duration_secs','error_count','is_beta_feature']] = master_df[['usage_count','usage_duration_secs','error_count','is_beta_feature']].fillna(0)

In [398]:
# Merge Aggregated Support Metrics
master_df = master_df.merge(ticket_summary, on='account_id', how='left')

# Fill missing support metrics with 0
master_df['total_tickets'] = master_df['total_tickets'].fillna(0)
master_df['resolution_time_hours'] = master_df['resolution_time_hours'].fillna(0)

In [399]:
# handle missing satisfaction score with median
median_satisfaction = master_df['satisfaction_score'].median()
master_df['satisfaction_score_filled'] = master_df['satisfaction_score'].fillna(median_satisfaction)

In [400]:
# merge churn events
master_df = master_df.merge(churn_summary, on='account_id', how='left')

In [401]:
# Calculate Tenure in Months
# Create a new column for filled end_date (observation_end)
today = pd.Timestamp.today()
master_df['end_date_filled'] = master_df['end_date'].fillna(today)

# Recalculate tenure in months using filled end_date
master_df['tenure_months_filled'] = (master_df['end_date_filled'] - master_df['start_date']).dt.days / 30

# Quick check
master_df[['subscription_id', 'start_date', 'end_date', 'end_date_filled', 'tenure_months_filled', 'churn_flag']].head(10)

# check active vs churned subscriptions
active = master_df[master_df['churn_flag'] == False]
churned = master_df[master_df['churn_flag'] == True]

print(f"Active subscriptions: {active.shape[0]}")
print(f"Churned subscriptions: {churned.shape[0]}")

Active subscriptions: 4514
Churned subscriptions: 486


In [402]:
# handle missing churn info
# fill active subscriptions for clarity
master_df['reason_code_filled'] = master_df['reason_code'].fillna('Active')
master_df['feedback_text_filled'] = master_df['feedback_text'].fillna('No feedback')

In [403]:
# Churn indicator
master_df['Churned'] = master_df['churn_flag'].astype(int)

In [404]:
# Create cohort column
master_df['signup_month'] = master_df['signup_date'].dt.to_period('M')

In [405]:
# Drop duplicate columns if any remain
master_df = master_df.loc[:,~master_df.columns.duplicated()]

In [406]:
# Overwrite original columns with _filled versions
master_df['end_date'] = master_df['end_date_filled']
master_df['tenure_months'] = master_df['tenure_months_filled']
master_df['satisfaction_score'] = master_df['satisfaction_score_filled']
master_df['reason_code'] = master_df['reason_code_filled']
master_df['feedback_text'] = master_df['feedback_text_filled']

# Drop the _filled columns to clean up
master_df.drop(columns=['end_date_filled','tenure_months_filled',
                        'satisfaction_score_filled','reason_code_filled',
                        'feedback_text_filled'], inplace=True)

# Quick check
master_df.isna().sum()

subscription_id          0
account_id               0
start_date               0
end_date                 0
plan_tier_x              0
seats_x                  0
mrr_amount               0
arr_amount               0
is_trial_x               0
upgrade_flag             0
downgrade_flag           0
churn_flag               0
billing_frequency        0
auto_renew_flag          0
industry                 0
country                  0
plan_tier_y              0
seats_y                  0
signup_date              0
referral_source          0
is_trial_y               0
usage_count              0
usage_duration_secs      0
error_count              0
is_beta_feature          0
total_tickets            0
resolution_time_hours    0
satisfaction_score       0
reason_code              0
feedback_text            0
Churned                  0
signup_month             0
tenure_months            0
dtype: int64

In [407]:
# Quick master df check 
print(master_df.shape)                     
print(master_df['subscription_id'].nunique()) 
master_df.info()
master_df.head()

(5000, 33)
5000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   subscription_id        5000 non-null   object        
 1   account_id             5000 non-null   object        
 2   start_date             5000 non-null   datetime64[ns]
 3   end_date               5000 non-null   datetime64[ns]
 4   plan_tier_x            5000 non-null   object        
 5   seats_x                5000 non-null   int64         
 6   mrr_amount             5000 non-null   int64         
 7   arr_amount             5000 non-null   int64         
 8   is_trial_x             5000 non-null   bool          
 9   upgrade_flag           5000 non-null   bool          
 10  downgrade_flag         5000 non-null   bool          
 11  churn_flag             5000 non-null   bool          
 12  billing_frequency      5000 non-null   object 

,subscription_id,account_id,start_date,end_date,plan_tier_x,seats_x,mrr_amount,arr_amount,is_trial_x,upgrade_flag,...,error_count,is_beta_feature,total_tickets,resolution_time_hours,satisfaction_score,reason_code,feedback_text,Churned,signup_month,tenure_months
0,S-8cec59,A-3c1a3f,2023-12-23,2024-04-12 00:00:00.000000,Enterprise,14,2786,33432,False,False,...,2.0,1.0,4.0,33.25,3.500000,Active,No feedback,1,2023-08,3.700000
1,S-0f6f44,A-9b9fe9,2024-06-11,2026-02-07 21:46:27.599192,Pro,17,833,9996,False,False,...,6.0,0.0,4.0,33.50,4.000000,features,switched to competitor,0,2023-09,20.200000
2,S-51c0d1,A-659280,2024-11-25,2026-02-07 21:46:27.599192,Enterprise,62,0,0,True,True,...,2.0,0.0,4.0,8.00,4.333333,pricing,switched to competitor,0,2023-11,14.633333
3,S-f81687,A-e7a1e2,2024-11-23,2024-12-13 00:00:00.000000,Enterprise,5,995,11940,False,False,...,5.0,0.0,1.0,45.00,4.000000,support,missing features,1,2024-06,0.666667
4,S-cff5a2,A-ba6516,2024-01-10,2026-02-07 21:46:27.599192,Enterprise,27,5373,64476,False,False,...,0.0,0.0,6.0,31.50,4.000000,budget,No feedback,0,2023-10,25.300000


In [408]:
# safe the cleaned data
master_df.to_csv('master_data.csv', index=False)

**Exploratory Data Analysis**

**1. Why Are Customers Leaving the Platform?**

In [409]:
# look at churned customers only
churned_df = master_df[master_df['Churned'] == 1]
churned_df.value_counts().sum()

np.int64(486)

Observation:
- 486 churned subscriptions out of 5000. This is roughly 9.7% churn rate
- Overall, the platform is retaining about 90% of customers, which is not bad, but churn is still worth analyzing.

In [410]:
# Count churn reasons 
churn_reason_counts = churned_df['reason_code'].value_counts()
churn_reason_percent = churned_df['reason_code'].value_counts(normalize=True) * 100

# Combine into one table
churn_reason_summary = pd.DataFrame({
    'count': churn_reason_counts,
    'percentage': churn_reason_percent
})

print(churn_reason_summary)

             count  percentage
reason_code                   
Active         134   27.572016
features        71   14.609053
support         66   13.580247
budget          66   13.580247
unknown         56   11.522634
pricing         48    9.876543
competitor      45    9.259259


Observations:

- “Active” (27.6%) – Many of these might be accounts that are still active 
**Top real churn reasons:**

- Features (14.6%). Customers left because they wanted missing or better features.
- Support (13.6%). Issues with customer support or response time.
- Budget (13.6%). Some customers could not afford the plan.

Other factors:

- Pricing (9.9%). Platform perceived as too expensive.
- Competitor (9.3%). Customers switched to another platform.
- Unknown (11.5%)- Reason not recorded.

Key takeaway:
- The major drivers of churn seem to be missing features, poor support, budget constraints, and competitors.

In [411]:
# feedback text
feedback_sample = churned_df[['reason_code', 'feedback_text']].dropna()
print(feedback_sample.head(10))

   reason_code     feedback_text
0       Active       No feedback
3      support  missing features
16    features       No feedback
17     support     too expensive
21    features  missing features
29      Active       No feedback
52      Active       No feedback
54  competitor     too expensive
64      Active       No feedback
73      Active       No feedback


Observation:
- Many entries for Active have “No feedback”, confirms earlier observation.
- Feedback like “missing features” and “too expensive” matches the reason codes for features, support, and budget.
- Feedback is consistent with reason codes, which is good for validation.

**Conclusion**

To reduce churn, the company should focus on:

- Improving features that customers need.

- Strengthening customer support and response times.

- Reviewing pricing plans for affordability.

- Monitoring competitor moves.

In [412]:
# Save for visuals later in power bi
churn_reason_summary.reset_index().rename(columns={'index':'reason_code'}).to_csv('churn_reason_summary.csv', index=False)

 **2. Which customer segments are most likely to churn?**

In [413]:
master_df.columns

Index(['subscription_id', 'account_id', 'start_date', 'end_date',
       'plan_tier_x', 'seats_x', 'mrr_amount', 'arr_amount', 'is_trial_x',
       'upgrade_flag', 'downgrade_flag', 'churn_flag', 'billing_frequency',
       'auto_renew_flag', 'industry', 'country', 'plan_tier_y', 'seats_y',
       'signup_date', 'referral_source', 'is_trial_y', 'usage_count',
       'usage_duration_secs', 'error_count', 'is_beta_feature',
       'total_tickets', 'resolution_time_hours', 'satisfaction_score',
       'reason_code', 'feedback_text', 'Churned', 'signup_month',
       'tenure_months'],
      dtype='object')

In [414]:
# Churn rate by plan_tier
plan_tier_churn = master_df.groupby('plan_tier_y')['Churned'].agg(['sum','count'])
plan_tier_churn['churn_rate_%'] = (plan_tier_churn['sum'] / plan_tier_churn['count']) * 100
plan_tier_churn = plan_tier_churn.sort_values('churn_rate_%', ascending=False)
print(plan_tier_churn)

             sum  count  churn_rate_%
plan_tier_y                          
Enterprise   161   1509     10.669317
Pro          175   1742     10.045924
Basic        150   1749      8.576329


Observation:

- Enterprise and Pro plans have slightly higher churn rates than Basic.
- This might seem counterintuitive since higher plans are more “sticky,” but it could indicate these customers have higher expectations and may leave if the platform doesn’t meet their needs.
- Basic plan customers are slightly more stable.

In [415]:
# Churn rate by industry
industry_churn = master_df.groupby('industry')['Churned'].agg(['sum','count'])
industry_churn['churn_rate_%'] = (industry_churn['sum'] / industry_churn['count']) * 100
industry_churn = industry_churn.sort_values('churn_rate_%', ascending=False)
print(industry_churn)

               sum  count  churn_rate_%
industry                               
DevTools       125   1158     10.794473
Cybersecurity  103   1008     10.218254
HealthTech      96    949     10.115911
FinTech         96   1100      8.727273
EdTech          66    785      8.407643


Observation:

- Customers in DevTools, Cybersecurity, and HealthTech have higher churn rates.
- FinTech and EdTech seem more loyal.
- This could reflect differences in adoption patterns, product fit, or competition in these industries.

In [416]:
# Churn rate by country
country_churn = master_df.groupby('country')['Churned'].agg(['sum','count'])
country_churn['churn_rate_%'] = (country_churn['sum'] / country_churn['count']) * 100
country_churn = country_churn.sort_values('churn_rate_%', ascending=False)
print(country_churn)

         sum  count  churn_rate_%
country                          
FR        25    211     11.848341
AU        35    312     11.217949
UK        64    590     10.847458
US       288   2961      9.726444
CA        21    217      9.677419
DE        20    211      9.478673
IN        33    498      6.626506


Observation:

- France, Australia, and the UK have higher churn rates than the US, Canada, and Germany.
- India shows the lowest churn, possibly due to fewer alternatives or higher satisfaction.
- This helps in regional prioritization for retention efforts.

In [417]:
# churn vs seats
bins = [0, 5, 20, 50, 100, 500]
labels = ['1-5','6-20','21-50','51-100','100+']
master_df['seats_bucket'] = pd.cut(master_df['seats_y'], bins=bins, labels=labels, include_lowest=True)

seats_churn = master_df.groupby('seats_bucket')['Churned'].agg(['sum','count'])
seats_churn['churn_rate_%'] = (seats_churn['sum'] / seats_churn['count']) * 100
print(seats_churn)

              sum  count  churn_rate_%
seats_bucket                          
1-5           147   1287     11.421911
6-20          180   1932      9.316770
21-50         124   1417      8.750882
51-100         31    325      9.538462
100+            4     39     10.256410


C:\Users\USER\AppData\Local\Temp\ipykernel_5344\3453343666.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  seats_churn = master_df.groupby('seats_bucket')['Churned'].agg(['sum','count'])


Observation:

- Small accounts (1–5 seats) have the highest churn.
- Mid-sized accounts (6–50 seats) are more stable.
- Very large accounts (100+ seats) have slightly higher churn again, maybe because large deployments are more complex and harder to manage.

✅ Summary of Customer Segments Most Likely to Churn:

By Plan: Enterprise & Pro

By Industry: DevTools, Cybersecurity, HealthTech

By Country: FR, AU, UK

By Account Size: Small accounts (1–5 seats) and very large accounts (100+ seats)

**3. How long do customers typically stay active?**

In [418]:
tenure_summary = master_df.groupby(['plan_tier_y', 'Churned'])['tenure_months'].agg(
    mean_tenure='mean',
    median_tenure='median',
    min_tenure='min',
    max_tenure='max',
    count='count'
).reset_index()

tenure_summary

,plan_tier_y,Churned,mean_tenure,median_tenure,min_tenure,max_tenure,count
0,Basic,0,18.813446,17.233333,13.433333,37.500000,1599
1,Basic,1,1.899333,0.916667,0.000000,11.733333,150
2,Enterprise,0,19.265653,17.666667,13.433333,37.400000,1348
3,Enterprise,1,3.224845,1.700000,0.000000,15.966667,161
4,Pro,0,19.131483,17.566667,13.433333,36.000000,1567
5,Pro,1,3.556571,1.966667,0.000000,20.800000,175


Observations:
#### Active customers (Churned = 0):
- Customers who are still active stay much longer than churned ones.
- Average tenure is roughly 18–19 months across all plan tiers.
- Median tenure is around 17–18 months, showing most active customers have been subscribed for over a year.

#### Churned customers (Churned = 1):
- Churned customers leave quickly, with mean tenure around 2–3.5 months.
- Median tenure is below 2 months, indicating most churn happens early in the subscription period.
- Maximum tenure for churned customers is under 21 months, much lower than active customers.

Plan tier comparison:

- Enterprise customers who churned tend to stay slightly longer (mean ≈ 3.22 months) than Basic customers (mean ≈ 1.9 months), but they still leave much earlier than active accounts.

- Pro churned customers have a slightly higher mean tenure (≈ 3.56 months) than Basic and Enterprise churned accounts, showing some differences by plan tier.

Key takeaway:

- Most churn happens within the first 1–4 months of subscription, regardless of plan tier. Plan tier slightly affects how long churned customers stay, but the difference isn’t huge.

**4. What actions can improve retention?**

1. Address Key Churn Reasons

From reason_code analysis:
- Active / No feedback (27.6%): Customers may have lost interest or forgot about the product.

Action: Implement regular engagement emails, product tips, and reminders.

- Missing Features (14.6%):

Action: Collect feedback on most requested features; prioritize feature updates.

- Support issues (13.6%):

Action: Improve customer support quality and response time; possibly add live chat or self-help resources.

- Budget / Pricing (23.5% combined):

Action: Offer tiered plans, discounts, or flexible payment schedules.

Competitor switching (9.3%):

Action: Conduct win-back campaigns, highlight unique value, and track competitor offerings.

2. Focus on High-Risk Segments

From churn rate by segment:

- Plan tiers: Enterprise and Pro have slightly higher churn rates (~10%) than Basic (~8.6%).

- Industries: DevTools and Cybersecurity show higher churn (~10–10.8%).

- Countries: FR, AU, and UK have highest churn rates (~11–12%).

- Seat buckets: Small accounts (1–5 seats) churn the most (~11.4%).

Actions:

- Tailored retention programs for high-risk segments:

- Small businesses: personalized onboarding and check-ins.

- High churn industries: targeted support, webinars, and product demos.

- High-churn regions: localized support, localized content, and promotions.

3. Act Early

From tenure analysis, most churn happens within the first 1–4 months.

Action:

- Focus retention efforts early: onboarding emails, tutorials, check-ins within the first 30–60 days.

- Monitor early usage patterns: low engagement = proactive outreach.

4. Improve Product Engagement

Track usage metrics: usage_count, usage_duration_secs, error_count.

Action:

- Encourage active usage of features, guide customers toward high-value workflows.

- Reduce friction and errors to improve satisfaction.

5. Feedback Loops

Many churned accounts had no feedback.

Action:

- Collect continuous feedback, even for active users.

- Use surveys, in-app prompts, or NPS (Net Promoter Score) to identify dissatisfaction early.

**Conclusion**

1. Why Customers Are Leaving

The top reasons for churn are:

(i) Inactive / No feedback (28%): customers lose interest or forget about the platform.

(ii) Missing features (15%) and support issues (14%): product limitations and poor support drive churn.

(iii) Budget & pricing concerns (23% combined) and switching to competitors (9%) also contribute.

(iv) Many churned users provide little or no feedback, limiting insight into their experience.

2. Which Segments Are Most at Risk

(i) Plan tier: Enterprise and Pro accounts show higher churn (~10%) than Basic (~8.6%).

(ii) Industry: DevTools and Cybersecurity customers have slightly higher churn (~10–11%).

(iii) Region: Customers from FR, AU, and UK have the highest churn rates (~11–12%).

(iv) Account size: Small accounts (1–5 seats) are more likely to churn (~11%).

3. Customer Tenure Insights

(i) Most churn happens within the first few months of subscription:

(ii) Median tenure for churned customers is <2 months.

(iii) Longer-tenure accounts (>12 months) are much less likely to churn.

4. Key Retention Opportunities

(i) Early engagement: onboarding, tutorials, and check-ins during the first 1–3 months.

(ii) Address product gaps: prioritize missing features and fix support issues.

(iii) Flexible pricing & incentives: target small accounts and price-sensitive segments.

(iv) Segment-focused retention: high-risk industries and regions get specialized campaigns.

(v) Continuous feedback loops: encourage feedback from both active and churned customers.

**Overall Takeaway**

**Churn is mostly early-stage and preventable. Companies that proactively engage new users, resolve support and feature gaps, and tailor retention strategies to high-risk segments can significantly reduce churn and improve lifetime value.**